In [5]:
# import modules 
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget


# Methodology

The Brillouin zones are determined by applying the Wigner-Seitz construction to the reciprocal lattice. Zone interiors are defined by the number of times a Bragg plane is crossed when traveling in a straight line from the origin. At present, only the first Brillouin zone is calculated, which simplfies matters because it consists of a single polyhedron. The polyhedron is defined in terms of its faces, each corresponding to a single Bragg plane or $\mathbf{K}$-point in the reciprocal lattice. The faces are in turn defined as a set of edges, each of which is defined as a set of two vectors denoting its start and end points. The 1st Brillouin zone is defined in an iterative manner: first, all Bragg planes (in a reasonably large volume of k-space) are ranked according to their distance from the origin; an initial cell is constructed using the set of such planes nearest to the origin; then the next nearest set of planes is used to truncate the cell, and this process is repeated for each successively more distant set of Bragg planes until it is no longer possible to do so. Whether to continue or not can be determined using the "vertex test:" if the distance to the farthest-out vertex of the polyhedron exceeds the distance to the family of Bragg planes in question, the planes may (though not necessarily will) intersect and therefore truncate the polyhedron.

This process is explained in more detail, with reference to relevant sections of the code:

## 0. Initialize Parameters
Define key constants for the construction - how many $\mathbf{K}$-points in the reciprocal lattice to include (i.e., what range of $\mathbf{k}$-space are we allowing in the calculations), how many zones to calculate, what the threshold for roundoff error should be when locating intersections of lines & planes. Also choose a real space lattice to define the crystal structure (simple cubic, fcc, bcc, hcp, etc.).

## 1. Generate the Reciprocal Lattice and Bragg Planes

### 1.1 Reciprocal Lattice

The reciprocal lattice is the set of all points in $\mathbf{k}$-space that are integer combinations of the reciprocal lattice vectors $\mathbf{b_1, b_2, b_3}$: $\{\mathbf{K} \in \mathbb{K}^3: \mathbf{K} = n_1\mathbf{b_1} + n_2\mathbf{b_2} +n_3\mathbf{b_3}, n_1, n_2, n_3 \in \mathbb{Z}\}$. The reciprocal lattice vectors are calculated from the real-space lattice vectors $\mathbf{a_1, a_2, a_3}$ using the definition $\mathbf{a}_i \cdot \mathbf{b}_j = \delta_{ij}$ (for simplicity, we do not employ the solid-state physics convention of including the factor $2\pi$). Hence,

$\displaystyle \mathbf{b_1 = \frac{a_2 \times a_3}{a_1 \cdot (a_2 \times a_3)}};
\qquad \mathbf{b_2 = \frac{a_3 \times a_1}{a_1 \cdot (a_2 \times a_3)}} ;
\qquad \mathbf{b_3 = \frac{a_1 \times a_2}{a_1 \cdot (a_2 \times a_3)}}$

For a restricted set of values of the integers $n_1, n_2, n_3$, we can generate a large enough subset of the full reciprocal lattice to determine the desired zones. This may not be known a priori, so it is probably safest to set the $n_\mathrm{max}$ parameter to be significantly larger than the desired number of zones to be calculated.

### 1.2 Bragg Planes

The Bragg planes are defined as the planes bisecting the vector $\mathbf{K}$ from the origin to specific point in the reciprocal lattice. The general description of the Bragg plane $\mathscr{P}_\mathbf{K}$ corresponding to lattice point $\mathbf{K}$ is

$\displaystyle \mathscr{P}_\mathbf{K} = \left\{ \mathbf{k} \in \mathbb{K}^3: \mathbf{k} = \frac{1}{2}\mathbf{K} + t\mathbf{v}, t \in \mathbb{R}, \mathbf{v} \in \{\mathbf{k} \in \mathbb{K}^3: \mathbf{k} \cdot \mathbf{K} = 0 \} \right\}$;

that is, points in the plane are defined by the "normal" vector $\mathbf{K_\frac{1}{2}} = \frac{1}{2}\mathbf{K}$ and an arbitrary vector $t\mathbf{v}$ that lies within the plane. Note that it is more convenient to work with the vectors $\mathbf{K}$ or $\mathbf{K_\frac{1}{2}}$ than the strict normal to the plane $\mathbf{\hat{n}_\mathscr{P_\mathbf{K}}} = \mathbf{\frac{K}{||K||}}$. The magnitude $||\mathbf{K_\frac{1}{2}}|| = \frac{1}{2}K$ represents the distance of closest approach of $\mathscr{P}_\mathbf{K}$ to the origin. The planes may then be conveniently ranked by the magnitude of their corresponding $\mathbf{K}$-points, and the results collected in a Pandas dataframe for easy reference.

## 2. Calculate Brillouin Zones

## 2A. Calculate the 1st Brillouin Zone

The first zone is special because it has no interior bounding surface(s), so we calculate it separately. The overall procedure will be to iterate through families of Bragg planes of increasing distance from the origin and use them to define a polyhedron enclosed by them and the previous cell. In other words, the first construction will create a trial cell, and successive iterations will create new cells by chopping off the corners of the previous one, until the next family of planes is too distant to do so.

### 2A.1. Determine Trial Cell

The first cell will be defined as the polyhedron whose faces are defined by the intersections of each Bragg plane with its nearest neighbors. Identification of the nearest neighbors is not trivial, and investigating the intersections of a given plane with all others in the same family do not sufficiently constrain the space, although this is a useful starting point.

Before doing so, it is useful to formally define what we mean by a polyhedron. For the present purposes, it suffices to describe a polyhedron as the surface $\mathscr{S}$ of all $\mathbf{k}$-points belonging to a set of $N_\mathrm{F}$ polygonal faces ${\mathscr{F}_i}$:

$\displaystyle \mathscr{S} = \left\{\mathbf{k} \in \mathbb{K}^3: \mathbf{k} \in \mathscr{F}_i, i = 1,2,...N_\mathrm{F} \right\} = \bigcup\limits_{i=1}^{N_\mathrm{F}} \mathscr{F}_i$

$\mathscr{N.B.}$ This definition obliges us to adopt a recursive scheme wherein each component of the polyhedron is defined by the bounding geometries of lower dimension - i.e., the 3D volume of the polyhedron is defined in terms of its bounding faces, each 2D face is defined by its bounding edges, and each 1D edge is a line segment defined by a pair of (0D) vertices. While it would be convenient to define the polyhedron solely as a list of its vertices, doing so is not practical because vital topological information is lost. This information is, however, preserved when the edges are listed, so it is best to adopt the hierarchical definition scheme described above.

Following the above scheme, each polygonal face $\mathscr{F}_i$ is defined by (though not rigorously equivalent to) the set of its $N_\mathrm{E}$ bounding edges $\{\mathscr{E}_{ij},j=1,2,...,N_\mathrm{E}\}$. Each edge is in turn defined by (though again, not rigorously equivalent to) the pair of points $(\mathbf{p}_{ij1},\mathbf{p}_{ij2})$ bounding the line segment. This scheme can be implemented in the code by defining the cell as a list of "face" objects. Each face is most conveniently represented as a dictionary with two key-value pairs: the "Edges" key corresponds to a list of pairs of vectors defining the edge endpoints; and the "K" key corresponds to the $\mathbf{K}$-point generating the Bragg plane in which the face lies. This information is not strictly necessary (the $\mathbf{K}$-point could be inferred from the edges), but it simplifies the job of keeping track of the faces.

#### 2A.1.1. Construct Individual Faces in the Trial Cell

We begin constructing the trial cell by looping over all $\mathbf{K}$-points defining the individual Bragg plane, and building the corresponding face by identifying the smallest area enclosed by all possible edges formed by the intersections of each other Bragg plane in the family.

##### 2A.1.1.1. Determine $\mathbf{K}$-Plane Intersections

To find the edges, we start by defining the lines of which the edges are a part, from the plane intersections. While scanning over all other planes, we can immediately reject any that are parallel, using the condition $\mathbf{K}_1 \times \mathbf{K}_2 = 0$, where $\mathbf{K}_1$ refers to the face-defining $\mathbf{K}$-point and $\mathbf{K}_2$ to the (possibly) edge-defining candidate plane; both are normal to their respective Bragg planes (but not of unit length). 

###### 2A.1.1.1.1. Define Bounding Lines

The general expression for any line is $\mathbf{p} + t\mathbf{v}$, where $\mathbf{p}$ is any point on the line, $\mathbf{v}$ is any vector parallel to the line, and $t \in \mathbb{R}$ is a parameter quantifying the distance along it. It is convenient to take $\mathbf{v} = \mathbf{K}_1 \times \mathbf{K}_2$, and to define $\mathbf{p}$ as the point of closest approach of the line to the origin, such that $\mathbf{p} \cdot \mathbf{v} = 0$. The components of $\mathbf{p}$ are further constrained by the requirement that it lie on both Bragg planes. Taking the first plane as an example, we can express $\mathbf{p} = \frac{1}{2}\mathbf{K}_1 + \mathbf{u}$ for an arbitrary vector $\mathbf{u}$ parallel to the plane (i.e., $\mathbf{u \cdot K_1} = 0$). Taking the dot product of both sides of the equation with $\mathbf{K}_1$ yields the relation $\mathbf{p \cdot K_1} = \frac{1}{2} \mathbf{K}_1 \cdot \mathbf{K}_1 = \frac{1}{2} K_1^2$. An equivalent expression is obtained for the other plane, so the components of p are completely specified by the projections of $\mathbf{p}$ onto $\mathbf{K}_1$, $\mathbf{K}_2$, and $\mathbf{v}$:

$\displaystyle \left\{ \begin{array}  \mathbf{\mathbf{p}} \cdot \mathbf{K}_1 =\frac{1}{2} K_1^2 \\ \mathbf{p} \cdot \mathbf{K}_2 =\frac{1}{2} K_2^2 \\ \mathbf{p} \cdot \mathbf{v} = 0 \\ \end{array} \right. $

This system of equations can be solved by forming the matrix $\mathsf{K}$ whose rows are $\mathbf{K}_1$, $\mathbf{K}_2$, and $\mathbf{v}$. Then, $\mathbf{p}$ is defined by the equation $\mathsf{K}\mathbf{p} = \mathbf{b}$, where $\mathbf{b} = (\frac{1}{2}K_1^2, \frac{1}{2}K_2^2, 0)^T$. Non-trivial solutions $\mathbf{p} = \mathsf{K}^{-1}\mathbf{b}$ exist if $\mathrm{det}\ \mathsf{K} \neq 0$, which is guaranteed by the nonvanishing cross product defining $\mathbf{v}$ in the parallel-plane filtering condition enforced above. Thus, we can define the bounding lines generated by each other Bragg plane by the pair of (vector-valued) parameters $\mathbf{p}$ and $\mathbf{v}$, and track them in a list.

###### 2A.1.1.1.2. Rank Bounding Lines by Distance from the Origin

Once the bounding lines are defined, they may be ranked by their distance of closest approach to the "center" of the Bragg plane - i.e., the point $\frac{1}{2} \mathbf{K}$ at which the vector $\mathbf{K}$ strikes it. An advantage of the above bounding line parameterization is that the point $\mathbf{p}$ is automatically the point on the bounding line closest to $\frac{1}{2} \mathbf{K}$, as well as the origin. (To prove this, minimize the magnitude of the general displacement vector $\frac{1}{2}\mathbf{K} - (\mathbf{p} + t\mathbf{v})$ to a point on the bounding line, which occurs at $t=0$.) Thus, the bounding lines may be conveniently partitioned into subsets according to the magnitudes $||\frac{1}{2}\mathbf{K} - \mathbf{p}||$ and ranked accordingly.

###### 2A.1.1.1.3. Construct Edges from Bounding Line Intersections

Although the bounding lines are now ranked, we still do not know which of the subsets are necessary to define the face, and which are superfluous. To make this discrimination, we deploy a process analogous to the face truncation, to wit: start with the nearest set of bounding lines to the $\mathbf{K}$-line and calculate the line segments corresponding to the edges, bounded by the intersections of those lines; then successively iterate through each successively more distant set of bounding lines, and apply the vertex test to determine whether it is possible to truncate the face further; if so, proceed; if not, then the face is complete. 

###### 2A.1.1.1.3.1. Specify the Initial Face

The edges of the trial face are determined by the line segments formed by the intersections of the "nearest-neighbor" bounding lines. We determine them by selecting each bounding line in turn ($\mathbf{k}_1 = \mathbf{p}_1 + t_1\mathbf{v}_1$), then looping through each other line in the family ($\mathbf{k}_2 = \mathbf{p}_2 + t_2\mathbf{v}_2$). The intersections with all other (non-parallel) bounding lines define a set of candidate vertices for the edge, of which only the two that are closest to the $\mathbf{K}$-line should be retained.

We first filter pairs of non-intersecting lines by applying the parallelism criterion $\mathbf{v}_1 \times \mathbf{v}_2 = 0$ and retaining only those pairs with non-vanishing cross product. Note, however, that this condition is not strong enough to guarantee intersection of the two lines, since they may have different "velocity" vectors but lie in parallel planes - i.e., be skew lines. The intersection criterion $\mathbf{p}_1 + t_1\mathbf{v}_1 = \mathbf{p}_2 + t_2\mathbf{v}_2$ is an overdetermined system of 3 equations in the two unknowns $(t_1, t_2)$, which may or may not have a solution. However, we can always calculate the minimum distance between the two lines, and classify them as skew or intersecting depending on whether this distance is finite or zero. The corresponding parameters must satisfy the relations

$\displaystyle \frac{\mathrm{d}}{\mathrm{d}t_1}[d] = \frac{\mathrm{d}}{\mathrm{d}t_2}[d] = 0\ , \qquad d = ||\mathbf{p_1 - p_2} + t_1\mathbf{v_1} - t_2\mathbf{v_2}||$

It is equally valid and more convenient to minimize $d^2$. Enforcing minimization conditions for $d^2$ corresponding to those above generates another system of linear equations, of the form $\mathsf{V}\mathbf{t} = \mathbf{q}$, where

$\displaystyle \mathsf{V} = \left( \begin{matrix} \mathbf{v_1 \cdot v_1} & - \mathbf{v_1 \cdot v_2} \\ - \mathbf{v_1 \cdot v_2} & \mathbf{v_2 \cdot v_2} \end{matrix} \right)\ ; \qquad \mathbf{t} = \left( \begin{matrix} t_1 \\ t_2 \end{matrix} \right) ; \qquad \mathbf{q} = \left( \begin{matrix} -\mathbf{v_1 \cdot (p_1 - p_2)} \\ \mathbf{v_2 \cdot (p_1 - p_2)} \end{matrix} \right)$

Solutions $\mathbf{t} = \mathsf{V}^{-1}\mathbf{q}$ are guaranteed provided that $\mathrm{det}\ \mathsf{V} \neq 0$, which is satisfied by the non-vanishing cross product criterion above. In this manner, points of intersection may be determined by calculating the requisite point $\mathbf{p}_1 + t_1\mathbf{v}_1 = \mathbf{p}_2 + t_2\mathbf{v}_2$.

Once a set of all such points formed by the intersections of the other bounding lines are determined, they may be ranked by distance to the $\mathbf{K}$-line, and the two nearest retained to determine the corresponding edge. 

After this process has been repeated for each bounding line in the family, we will have identified a set of $N_\mathrm{E}$ edges defining the perimeter of the initial face. Letting $\mathscr{F}$ denote the trial face, this boundary may be expressed as

$\displaystyle \mathscr{B}_\mathscr{F} = \partial \mathscr{F} = \left\{ \mathscr{E}_i, i = 1, 2, ..., N_\mathrm{E} : \mathscr{E}_i = \left\{ \mathbf{k} \in \mathbb{K}^3 : \mathbf{k} = \mathbf{p}_1^i + t(\mathbf{p}_2^i - \mathbf{p}_1^i), t \in [0,1] \right\} \right\}$

The boundary of each such edge may be expressed as the pair of points

$\displaystyle \mathscr{B}_{\mathscr{E}_i} = \partial \mathscr{E}_i = \{\mathbf{p}_1^i, \mathbf{p}_2^i\}$,

whose order is arbitrary. Once this family of edges has been specified, we may proceed to investigate whether more distant edges constrain the cell further.

###### 2A.1.1.1.3.2. Truncate the Face with Progressively More Distant Bounding Lines

Taking the face as defined above, we must now determine which edges, if any, are intersected by the next-most distant family of bounding lines. To determine whether or not to proceed, we apply the vertex test, comparing the distance to the $\mathbf{K}$-line to the furthest-out vertex against that of the nearest approach of the bounding lines, and proceed only if the former exceeds the latter. This condition assures that at least two edges will be intersected by the bounding line, and that a new cell will be formed by a) any non-intersected edges on the near side of the bounding line to the $\mathbf{K}$-line; b) the portions of the intersected edges on the near side of the bounding line to the $\mathbf{K}$-line; and c) the new edge defined by the two points of intersection.

For each bounding line $\mathscr{L}$ in the family, we must scan through all edges in the trial face and classify them according to the scheme above. We first discriminate between those that intersect and those that do not by defining the point of intersection between $\mathscr{L}$ and the line $\mathscr{L}_\mathscr{E}$ defining the edge. We first reject any lines that are parallel to the edge, using the parameterization $\mathbf{k} = \mathbf{p} + t\mathbf{v}$ for points on $\mathscr{L}$ and $\mathbf{k} = \mathbf{p}_1 + s (\mathbf{p}_2 - \mathbf{p}_1)$ for points on $\mathscr{L}_\mathscr{E}$: if $\mathbf{v} \times (\mathbf{p}_2 - \mathbf{p}_1) = 0$, we may reject $\mathscr{L}$ from further consideration. Otherwise, we determine the point of intersection $\mathbf{q}$ by setting the above expressions equal to each other:

$\mathbf{q} = \mathbf{p} + t\mathbf{v} = \mathbf{p}_1 + s (\mathbf{p}_2 - \mathbf{p}_1)$

Exploiting the parameterization of $\mathscr{L}$ such that $\mathbf{p \cdot v} = 0$, we take the dot product of both sides with $\mathbf{p}$: 

$\mathbf{p \cdot p} = \mathbf{p}_1 \cdot \mathbf{p} + s (\mathbf{p}_2 - \mathbf{p}_1) \cdot \mathbf{p}$

Rearranging,

$\displaystyle s = \mathbf{ \frac{(p - p_1) \cdot p}{(p_2 - p_1) \cdot p} }$

If $s \in (0,1)$, the bounding line strikes the interior of the line segment defining the edge (note that the open interval corresponds to the fact that a bounding line intersecting one of the endpoints will not truncate the edge), and we make note of $\mathbf{q}$ as one endpoint of the new edges formed by the bounding line (i.e., both the new edge lying on the line itself, and the new edge formed by truncation of the intersected edge). We next determine which of the endpoints of the edge will be retained to determine the new truncated edge, and which will be discarded. To determine which point is nearest to the $\mathbf{K}$-line, we test each by defining a parameterization of the line from $\frac{1}{2}\mathbf{K}$ to the point in question (say, $\mathbf{p}_1$, for which $\mathbf{k} = \frac{1}{2}\mathbf{K} + s_1(\mathbf{p}_1 - \frac{1}{2}\mathbf{K})$), and calculate the point at which this line intersects the bounding line $\mathscr{L}$, retaining the above parameterization. Using the same logic as above, $\mathbf{p}_1$ only lies on the far side of $\mathscr{L}$ if $\mathscr{L}$ strikes the line segment between $\mathbf{p}_1$ and $\frac{1}{2}\mathbf{K}$. That is, if the parameter

$\displaystyle s = \mathbf{ \frac{(p - \frac{1}{2}\mathbf{K}) \cdot p}{(p_1 - \frac{1}{2}\mathbf{K}) \cdot p} }$

lies in the interval $(0,1)$, the point $\mathbf{p}_1$ lies on the far side of $\mathscr{L}$ and should be rejected in favor of $\mathbf{p}_2$, and vice versa if it takes on any negative value, or positive value >1. (Strict equality with 1 implies that $\mathbf{p}_1$ lies on $\mathscr{L}$, which cannot be used for classification).

Having identified the proper point to retain, the newly truncated edge is fully specified and is retained in the list of truncated edges.

If $s \notin (0,1)$, or the bounding line is parallel to the edge, we use the classification test above to determine whether $\mathbf{p}_1$ and $\mathbf{p}_2$ are on the near or far side of $\mathscr{L}$, retaining the edge if the former case holds and discarding it if not.

Once all edges have been tested, the new face may be specified by collecting all of the near-side edges, the freshly truncated edges, and the single new edge. This process is repeated for all bounding lines in the family until the face can no longer be truncated, and the whole process is repeated for each other face until the full cell is constructed.



In [16]:
##### 0. Define parameters

n_max = 3 # how many points in k-space to investigate from the origin 
n_zones = 1 # how many zones to plot
eps_roundoff = 1e-10 # round-off error margin

make_plot = True # display plots or not
debug = True # toggle debug mode

# Cartesian unit vectors
xhat = np.array([1,0,0])
yhat = np.array([0,1,0])
zhat = np.array([0,0,1])

# define the lattice vectors in real space

# simple cubic
# a1 = xhat
# a2 = yhat
# a3 = zhat

# face-centered cubic
# a1 = 0.5*(yhat + zhat)
# a2 = 0.5*(zhat + xhat)
# a3 = 0.5*(xhat + yhat)

# body-centered cubic
# a1 = 0.5*(-xhat + yhat + zhat)
# a2 = 0.5*(xhat - yhat + zhat)
# a3 = 0.5*(xhat + yhat - zhat)

# hexagonal - UNDER DEVELOPMENT
# a1 = 0.5*xhat - np.sqrt(3)/2*yhat
# a2 = 0.5*xhat + np.sqrt(3)/2*yhat
# a3 = zhat

##### 1. Generate the reciprocal lattice and assign Bragg planes

##### 1.1. Reciprocal Lattice

# calculate the reciprocal vectors

trip_prod = np.dot(a1,np.cross(a2,a3)) # lattice vector triple product

# reciprocal basis
b1 = (1/trip_prod)*np.cross(a2,a3)
b2 = (1/trip_prod)*np.cross(a3,a1)
b3 = (1/trip_prod)*np.cross(a1,a2)

# calculate K-points in the reciprocal lattice

# initialize list of "quantum numbers"
n1s = []
n2s = []
n3s = []
Ks = [] # K-points, as vectors
K_mags = [] # magnitudes of K-vectors

# loop over indices to populate "quantum numbers" and K-vectors
for n1 in range(-n_max,n_max,1):
    for n2 in range(-n_max,n_max,1):
        for n3 in range(-n_max,n_max,1):
            n1s.append(n1)
            n2s.append(n2)
            n3s.append(n3)
            Ks.append(n1*b1 + n2*b2 + n3*b3)
            K_mags.append(np.linalg.norm(n1*b1 + n2*b2 + n3*b3))

# create a dataframe to keep track of the K-points and their distances from the origin
K_DF = pd.DataFrame({'K':Ks,
                     'n1':n1s,
                     'n2':n2s,
                     'n3':n3s,
                     '|K|':K_mags,
                    })

# assign "distances" from the origin
K_DF['n^2'] = K_DF['n1']**2 + K_DF['n2']**2 + K_DF['n3']**2

##### 1.2 Define Bragg Planes

# sort the entries according to distance
K_DF = K_DF.sort_values('|K|',ignore_index=True)

# rank the planes by distance
ranks = []
rank = 0
for ii in range(len(K_DF['|K|'].values)):
    if ii == 0:
        ranks.append(rank)
        continue
    elif np.abs(K_DF['|K|'].values[ii] - K_DF['|K|'].values[ii-1]) > eps_roundoff:
        rank += 1
    ranks.append(rank)
K_DF['Distance Index'] = ranks

##### 2. Calculate Brillouin zones

# loop over zone indices
for zonem1 in range(n_zones):
    
    zone = zonem1 + 1 # begin zone counter at 1
    
    ##### 2A. Calculate the 1st Brillouin zone
    
    # 1st zone is special because there are no interior bounding zones
    if zone == 1:
        
        ##### 2A.1 Initialize Trial Cell
        
        # set up list of faces that define the first Brillouin zone
        BZ1_faces = []
        
        # set up figure and axes to display the initial cell
        if make_plot:
            z_fig = plt.figure(figsize=(10,10))
            ax = z_fig.add_subplot(projection='3d')
        
        # now determine which pairs of planes intersect, pulling only the subset of K-planes nearest the origin 
        K_red = K_DF[K_DF['Distance Index'] == zone] 
        idx_list = list(K_red.index) # convert indices to list to enable list comprehension
        
        ##### 2A.1.1. Construct Individual Faces in Trial Cell
        
        # scan over each K-point, corresponding to a face in the cell
        for ii in idx_list:

            if debug:
                print('Constructing face',ii)

            # initialize the dictionary and variables relating to face definition:
            face = {} # initialize an empty dictionary
            face['K'] = K_red['K'][ii] # assign the corresponding K point
            bounding_lines = [] # initialize a list of parameters to find the lines bounding that face
            edges = [] # initialize a list of vector pairs to describe edges

            # generate a list of indices of all other planes
            red_idx_list = [item for item in idx_list if item != ii]

            # scan over all other K-planes of equivalent distance to find which ones intersect and if so,
            # define the line of their intersection
            for jj in red_idx_list:
                
                ##### 2A.1.1.1 Determine Plane Intersections

                # if the planes are not parallel:
                if np.linalg.norm(np.cross(K_red['K'][ii],K_red['K'][jj])) != 0:

                    # find the corresponding line of intersection between the planes
                    K1 = K_red['K'][ii] # K-point 1
                    K2 = K_red['K'][jj] # K-point 2
                    v = np.cross(K1,K2) # "velocity" vector lying along the line

                    # any point on the line will do, but for simplicity we choose the point closest to the origin,
                    # so p (dot) v = 0. We can set up a matrix of coefficients and solve by inverting:
                    K_mat = np.array([K1,K2,v])
                    b = np.array([0.5*np.dot(K1,K1),0.5*np.dot(K2,K2),0])
                    p = np.matmul(np.linalg.inv(K_mat),b)

                    # keep track of the parameters defining the bounding lines
                    bounding_lines.append([p,v])

            # we now have a set of bounding lines, but only a subset will actually define the edges of the face.
            # We will rank them by distance to the point at which the plane is intersected by the line along K,
            # defined by the vector 1/2 K.
            
            ###### 2A.1.1.1.2. Rank Bounding Lines by Distance from the K-Line

            # convert the list to a dataframe
            BL_dict = {}
            BL_dict['Line Parameters'] = bounding_lines
            BL_DF = pd.DataFrame(BL_dict)
            K_distances = []

            # assign minimum distances as distance between 1/2 K and line definition point p... 
            for params in bounding_lines:
                K_distances.append(np.linalg.norm(params[0] - 0.5*K_red['K'][ii]))
            BL_DF['1/2 K Distance'] = K_distances
            # ...then sort in ascending order 
            BL_DF = BL_DF.sort_values('1/2 K Distance',ignore_index=True)

            # rank the lines by distance
            l_ranks = []
            for jj in range(len(BL_DF['1/2 K Distance'].values)):
                # first iteration: set rank = 1
                if jj == 0:
                    l_rank = 1
                # successive iterations: if distance changes beyond roundoff error, increment rank
                elif np.abs(BL_DF['1/2 K Distance'].values[jj] - BL_DF['1/2 K Distance'].values[jj-1]) > eps_roundoff:
                    l_rank += 1
                l_ranks.append(l_rank)
            BL_DF['Distance Index'] = l_ranks
            
            ###### 2A.1.1.1.3. Construct Edges from Bounding Line Intersections
            
            jj = 1
            chop = True

            # now iterate over each block of bounding lines, ranked by distance
            while chop:

                # pare down the bounding lines dataframe to only include lines of the selected distance
                BL_red = BL_DF[BL_DF['Distance Index'] == jj]
                BLs = BL_red['Line Parameters'].values # rename for brevity

                # on the first pass, calculate the initial face
                if jj == 1:

                    ###### 2A.1.1.1.3.1. Specify the Initial Face
                    
                    if debug:
                        print('Set',jj,'of bounding lines')

                    trial_edges = [] # initialize empty list of edges

                    # now scan over all bounding lines in the subset
                    for kk, params in enumerate(BLs):

                        edge_candidates = [] # initialize the list of potential endpoints defining an edge
                        center_distances = [] # and corresponding list of distances from the "K-line"

                        # generate a list of all the other lines in the subset
                        red_lines = list(BLs).copy()
                        red_lines.pop(kk)

                        # for each other line in the face:
                        for params2 in red_lines:
                            # extract parameters corresponding to the two lines
                            p1 = params[0]
                            v1 = params[1]
                            p2 = params2[0]
                            v2 = params2[1]

                            # reject parallel line pairs
                            if np.linalg.norm(np.cross(v1,v2)) == 0:
                                continue
                            else:    
                                # solve for the parameters (t-values) yielding the minimum distance between lines
                                V_mat = np.array([[np.dot(v1,v1), -np.dot(v1,v2)],
                                                  [-np.dot(v1,v2), np.dot(v2,v2)],
                                                 ])
                                q = np.array([-np.dot(v1,p1-p2),np.dot(v2,p1-p2)])
                                t = np.matmul(np.linalg.inv(V_mat),q)
                                
                                # calculate the displacement vector between them
                                line_disp = p1 - p2 + t[0]*v1 - t[1]*v2

                                # if the lines intersect, one of the endpoints has been found:
                                if np.linalg.norm(line_disp) < eps_roundoff:
                                    edge_candidates.append(p1 + t[0]*v1)
                                    center_distances.append(np.linalg.norm(p1 + t[0]*v1 - 0.5*K_red['K'][ii]))
                        
                        # depending on the complexity of the face, we may have discovered more than two intersections
                        # we now rank candidate points by distance from the K-line and retain only the two closest
                        
                        edge_dict = {} # compile the edge candidate and distance in a dictionary...
                        edge_dict['Point'] = edge_candidates
                        edge_dict['Distance'] = center_distances
                        edge_DF = pd.DataFrame(edge_dict) # ...convert to a dataframe...
                        edge_DF = edge_DF.sort_values('Distance',ignore_index=True) # ...and sort by distance
                        edge = edge_DF['Point'].values[:2] # define the edge using only the first two points
                        
                        trial_edges.append(edge) # after looking at all other bounding lines, keep track of the edges

                    for edge in trial_edges:
                        if make_plot:
                            t = np.linspace(0,1,num=2)

                            x = edge[0][0] + t*(edge[1][0] - edge[0][0])
                            y = edge[0][1] + t*(edge[1][1] - edge[0][1])
                            z = edge[0][2] + t*(edge[1][2] - edge[0][2])

                            ax.plot(x,y,z,'b--')
                            ax.set_title('Initial Cell')

    #                 if debug:
    #                     chop = False

                    jj += 1
                
                ###### 2A.1.1.1.3.2. Truncate the Face with Progressively More Distant Bounding Lines
            
                # on successively distant sets of bounding lines, try to truncate the face
                else:

                    # first, apply the "vertex test" to determine whether the new set of edges even pass close enough
                    # to the 1/2 K point to truncate the face
                    vert_dist = 0 # initialize max vertex distance
                    for edge in trial_edges: # scan over all points in the trial edge:
                        for point in edge:
                            # if we find a more distant vertex, record the distance
                            if np.linalg.norm(point - 0.5*K_red['K'][ii]) > vert_dist:
                                vert_dist = np.linalg.norm(point - 0.5*K_red['K'][ii])

                    # if truncation is not possible, raise the stop condition
                    if vert_dist - BL_red['1/2 K Distance'].iloc[0] < eps_roundoff:

                        edges = trial_edges
                        chop = False


                    # if we keep truncating:
                    else:    
                        # initialize variables to keep track of edges of the new face
                        new_edge = [] # generated by truncation
                        E_near = [] # on the near side of the K-point, retained unchanged
                        E_cut = [] # truncated edges
                        edges = trial_edges.copy()

                        # scan over bounding lines
                        for kk, params in enumerate(BLs):

                            # get parameters defining the bounding line
                            p = params[0]
                            v = params[1]

                            # scan over trial edges
                            for edge in edges:

                                # get parameters defining endpoints of the edge
                                p1 = edge[0]
                                p2 = edge[1]

                                # distance along the line to the bounding line intersection
                                s1 = np.dot(p-0.5*K_red['K'][ii],p) / np.dot(p1-0.5*K_red['K'][ii],p)
                                s2 = np.dot(p-0.5*K_red['K'][ii],p) / np.dot(p2-0.5*K_red['K'][ii],p)

                                # if the bounding line and edge are not parallel:
                                if np.dot(v,p2-p1) !=0 :
                                    
                                    # calculate the parameter defining how far along the edge line the bounding
                                    # line strikes it
                                    s = np.dot(p-p1,K_red['K'][ii]) / np.dot(p2-p1,K_red['K'][ii])

                                    # if the bounding line intersects the line segment (not just the line)...
                                    if s>0 and s<1:
                                        vert = p1 + s*(p2-p1) # calculate the new vertex
                                        new_edge.append(vert) # add it to the definition of the new edge

                                        # determine which of the original vertices is on the 1/2 K point side of
                                        # the bounding line by calculating the point at which the line through 1/2 K and
                                        # p1 strikes the bounding line

                                        # if the distance parameter is greater than 1 or is negative, 
                                        # p1 is on the near side
                                        if s1 > 1 or s1 < 0:
                                            E_cut.append([vert,p1])
                                        # otherwise, p2 is the near one
                                        else:
                                            E_cut.append([vert,p2])

                                    # if the bounding line is not parallel to the edge but does not intersect it either:
                                    else:
                                        # if the distance parameter is greater than 1 or is negative, 
                                        # p1 is on the near side
                                        if s1 > 1 or s1 < 0:
                                            E_near.append(edge)

                                # if the bounding line is parallel to the edge:
                                else:
                                    # if the distance parameter is greater than 1 or is negative, 
                                    # p1 is on the near side
                                    if (s1 > 1 or s1 < 0) and (s2 > 1 or s2 < 0):
                                        E_near.append(edge)    


                        # after truncating the edges, reconstruct the face-defining list
                        trial_edges = E_near + E_cut + [new_edge]

                        for edge in trial_edges:
                            if make_plot:
                                t = np.linspace(0,1,num=2)

                                x = edge[0][0] + t*(edge[1][0] - edge[0][0])
                                y = edge[0][1] + t*(edge[1][1] - edge[0][1])
                                z = edge[0][2] + t*(edge[1][2] - edge[0][2])

                                ax.plot(x,y,z,'k--')
                                ax.set_title('Initial Cell')

                        jj += 1

            face['Edges'] = edges # after looking at all bounding lines, add the set of edges to the face dictionary
            BZ1_faces.append(face) # and update the main list of faces





        # now that we have set up the first cell, we attempt to truncate with the next nearest set of Bragg planes
        chop = True # assume that the next set of planes will intersect the current cell
        ii = 1
        while chop:

            print('Testing planes of distance index',zone + ii)

            # first figure out if we are done chopping off cell corners by comparing the cell vertex farthest from the 
            # origin with the closest approach of the new set of Bragg planes to the origin -
            # if the latter exceeds the former, there is no way that the planes could truncate the cell, and so we are
            # done
            K_red = K_DF[K_DF['Distance Index'] == zone + ii] # take next set of Bragg planes, redefining "K-red"
            halfK = 0.5*np.linalg.norm(K_red['K'].values[0]) # calculate the minimum distance to the origin for the plane
            max_vert = 0 # initialize distance to the farthest vertex
            for face in BZ1_faces: # scan through all faces in the cell
                for edge in face['Edges']: # scan through all edges in the face
                    for vert in edge: # look at both vertices
                        if np.linalg.norm(vert) > max_vert: # if the vertex is farther than the old maximum,
                            max_vert = np.linalg.norm(vert) # make it the new maximum

            if max_vert < halfK: # if we are done, raise the stop condition
                chop = False
            else:                # otherwise, keep going
                # loop over planes in the new set
                idx_list = list(K_red.index)
                for jj in idx_list:

                    if debug:
                        print('jj = ', jj)

                    # yank the K-point corresponding to the plane
                    K = K_red['K'][jj]

                    # initialize 3 lists of faces
                    F_int = [] # faces intersected by the Bragg plane
                    F_near = [] # faces not intersected by the Bragg plane on the NEAR side of the origin - keep these
                    F_far = [] # faces not intersected by the Bragg plane on the FAR side of the origin - ignore these

                    # classify the faces in the cell into one of the three types above
                    for face in BZ1_faces:

                        assigned = False

                        # does the plane intersect the face? break the face down into edges
                        for edge in face['Edges']:

                            # define edge endpoints
                            p1 = edge[0]
                            p2 = edge[1]

                            # first, is the edge parallel to the plane?
                            if np.dot(p2-p1,K) == 0: # if yes, no intersection is possible and move on to next edge
                                continue
                            else:                    # if no: 
                                # calculate where the plane strikes the line relative to the edge
                                # t: distance along the line from p1 towards p2, as a fraction of the distance ||p2 - p1||
                                t = (0.5*np.dot(K,K) - np.dot(p1,K))/(np.dot(p2-p1,K))
                                if t>0 and t<1: # if t is in the interval (0,1), the edge is sectioned by the plane
                                                # note that t = 0 or t = 1 imply that the plane grazes a vertex,
                                                # which does not count as an intersection
    #                                 intersection == True
                                    F_int.append(face) # classify the face appropriately and break out of the edge loop
                                    assigned = True
                                    break


                        # if we are still going, we haven't found an intersecting edge, so the face goes in F2 or F3
                        # do this by picking a vertex in the face and seeing whether it is on the near or far side of the Bragg plane
                        # relative to the origin - if so, the whole face is on the same side because we have already established that 
                        # the plane does not intersect the face. In general, we should only need one, but if we happen to be unlucky
                        # because we chose a vertex that lies exactly on the plane, we can test others until we find one that isn't
                        if not assigned:
                            for edge in face['Edges']:
                                vert = edge[0] # pick an arbitrary vertex in the face - say first of first edge
                                # if this vertex is closer to the origin than its projection into the Bragg plane, the whole face is too
                                if 0.5*np.linalg.norm(K) - np.dot(vert,K/np.linalg.norm(K)) > 0:
                                    F_near.append(face) # add it to the list and stop looking
                                    break
                                # if this vertex is farther from the origin than its projection into the Bragg plane, the whole face is too
                                elif 0.5*np.linalg.norm(K) - np.dot(vert,K/np.linalg.norm(K)) < 0:
                                    F_far.append(face) # add it to the list and stop looking
                                    break
                                else:
                                    vert = edge[1]
                                    # if this vertex is closer to the origin than its projection into the Bragg plane, the whole face is too
                                    if 0.5*np.linalg.norm(K) - np.dot(vert,K/np.linalg.norm(K)) > 0:
                                        F_near.append(face) # add it to the list and stop looking
                                        break
                                    # if this vertex is farther from the origin than its projection into the Bragg plane, the whole face is too   
                                    elif 0.5*np.linalg.norm(K) - np.dot(vert,K/np.linalg.norm(K)) < 0:
                                        F_far.append(face) # add it to the list and stop looking
                                        break

                    # now that we have classified the faces, we redefine the polyhedron
                    BZ1_new = F_near.copy() # non-intersected faces on the near side of the Bragg plane are kept

                    edges_new = [] # each intersected face will contribute one edge to a new face

                    # DEBUGGING: plot the plane where it intersects the original cell
    #                 if debug:

    #                     d_fig = plt.figure()
    #                     d_ax = d_fig.add_subplot(projection='3d')

    #                     for old_face in BZ1_faces:
    #                         for old_edge in old_face['Edges']:
    #                             old_t = np.linspace(0,1,num=2)

    #                             old_x = old_edge[0][0] + old_t*(old_edge[1][0] - old_edge[0][0])
    #                             old_y = old_edge[0][1] + old_t*(old_edge[1][1] - old_edge[0][1])
    #                             old_z = old_edge[0][2] + old_t*(old_edge[1][2] - old_edge[0][2])

    #                             d_ax.plot(old_x,old_y,old_z,'k--')


    #                     x_plane = np.linspace(-max_vert,max_vert,num=101)
    #                     y_plane = np.linspace(-max_vert,max_vert,num=101)
    #                     XX, YY = np.meshgrid(x_plane,y_plane)

    #                     if np.dot(K,zhat) != 0:
    #                         ZZ = (0.5*np.dot(K,K) - K[0]*XX - K[1]*YY)/K[2]
    #                         d_ax.plot_surface(XX,YY,ZZ,alpha=0.2)

                    # scan through the intersected faces
                    for face in F_int:

                        # classify the edges in the same manner as the faces
                        E_int = [] # edges intersecting the plane - to be altered
                        E_near = [] # edges on the near side of the plane
                        E_far = [] # edges on the far side of the plane

                        new_face_edge = [] # ONE EDGE ONLY - defined by the points of intersection

                        for edge in face['Edges']:

                            # define edge endpoints
                            p1 = edge[0]
                            p2 = edge[1]

                            # calculate where the plane strikes the line relative to the edge
                            # t: distance along the line from p1 towards p2, as a fraction of the distance ||p2 - p1||
                            t = (0.5*np.dot(K,K) - np.dot(p1,K))/(np.dot(p2-p1,K))
                            if t>0 and t<1: # if t is in the interval (0,1), the edge is sectioned by the plane
                                            # note that t = 0 or t = 1 imply that the plane grazes a vertex,
                                            # which does not count as an intersection

                                # if the edge is intersected, initialize a new edge to be cut
                                new_edge = []

                                # make note of the vertex of intersection and append it to the new edge 
                                # and the edge for the new face
                                vert = p1 + t*(p2-p1)
                                new_face_edge.append(vert)
                                new_edge.append(vert)

                                # to reconstruct the new edge for the old face, figure out whether p1 or p2 is on the 
                                # origin side of the Bragg plane - keep the near one and discard the other
                                if 0.5*np.linalg.norm(K) - np.dot(p1,K/np.linalg.norm(K)) > 0:
                                    new_edge.append(p1)
                                else:
                                    new_edge.append(p2)

                                # append the truncated edge to the appropriate list
                                E_int.append(new_edge)

                            # for non-intersecting edges, use the vertex distance test to determine which side of the Bragg 
                            # plane the edge is on, and assign accordingly
                            else:
                                if (0.5*np.linalg.norm(K) - np.dot(p1,K/np.linalg.norm(K)) > 0) or (0.5*np.linalg.norm(K) - np.dot(p2,K/np.linalg.norm(K)) > 0):
                                    E_near.append(edge) # add it to the list and stop looking
                                if (0.5*np.linalg.norm(K) - np.dot(p1,K/np.linalg.norm(K)) < 0) or (0.5*np.linalg.norm(K) - np.dot(p2,K/np.linalg.norm(K)) < 0):
                                    E_far.append(edge) # add it to the list and stop looking

                        # after scanning over all the edges, both vertices of the edge for the new face should be assigned,
                        # so add the edge to the appropriate list
                        edges_new.append(new_face_edge)

                        # the newly truncated face can also be redefined
                        trunc_face = face.copy() # the new face inherits the K-point from the old one
                        trunc_face['Edges'] = E_near + E_int + [new_face_edge] # near edges + truncated edges + new one
                        BZ1_new.append(trunc_face) # finally, add it to the new list of faces

                    # after scanning over all the faces, complete the new cell by adding the face created by the truncation
                    new_face = {}
                    new_face['K'] = K # K-point corresponding to the chopping Bragg plane
                    new_face['Edges'] = edges_new # edges are all new ones generated by truncation

                    BZ1_new.append(new_face) # add the new face to the face list

                    BZ1_faces = BZ1_new # and now redefine the face list

    #                 chop = False # FOR DEVELOPMENT ONLY - prevents infinite loops

            ii += 1 # increment plane family counter 

        # plot the final cell after all truncations complete
        if make_plot:

            # set up figure/axes
            zf_fig = plt.figure(figsize = (10,10))
            f_ax = zf_fig.add_subplot(projection='3d')

            t = np.linspace(0,1,num=2) # edge length parameter

            # iterate over faces and edges
            for face in BZ1_faces:
                for edge in face['Edges']:

                    # calculate line segments corresponding to edges
                    x = edge[0][0] + t*(edge[1][0] - edge[0][0])
                    y = edge[0][1] + t*(edge[1][1] - edge[0][1])
                    z = edge[0][2] + t*(edge[1][2] - edge[0][2])

                    f_ax.plot(x,y,z,'r-') # plot the edge

            # annotate the plot
            f_ax.set_xlabel('$k_x$')
            f_ax.set_ylabel('$k_y$')
            f_ax.set_zlabel('$k_z$')
            f_ax.set_title('Final Construction')

            f_ax.set_box_aspect((1, 1, 1)) # equalize axis scale
    
    # in construction - calculate higher-order zones
    else:
        pass           
         

            

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Constructing face 1
Set 1 of bounding lines


IndexError: single positional indexer is out-of-bounds

In [4]:
%matplotlib widget

# creating figure
fig = plt.figure()
ax = Axes3D(fig)

     
for face in BZ1_faces:
    for edge in face['Edges']:

        t = np.linspace(0,1,num=2)

        x = edge[0][0] + t*(edge[1][0] - edge[0][0])
        y = edge[0][1] + t*(edge[1][1] - edge[0][1])
        z = edge[0][2] + t*(edge[1][2] - edge[0][2])

        ax.plot(x,y,z,'r--') 
            


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
K_red

,K,n1,n2,n3,|K|,n^2,Distance Index
1,"[0.0, 0.0, 1.0]",0,0,1,1.0,1,1
2,"[0.0, 0.0, -1.0]",0,0,-1,1.0,1,1


In [12]:
a1 = [1,0,0]
a2 = [0,2,0]
c = np.cross(a1,a2)

mat = np.array([a1,a2,c])
mat

array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 2]])

In [ ]:
b3

In [ ]:
np.dot(p2-p1,K)